# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770877165944                   -0.52    9.0         
  2   -2.772147167416       -2.90       -1.32    1.0    168ms
  3   -2.772170373763       -4.63       -2.48    1.0    151ms
  4   -2.772170654855       -6.55       -3.19    1.0    153ms
  5   -2.772170722088       -7.17       -3.96    2.0    194ms
  6   -2.772170722415       -9.49       -4.08    1.0    156ms
  7   -2.772170722991       -9.24       -4.88    1.0    164ms
  8   -2.772170723010      -10.73       -5.17    1.0    162ms
  9   -2.772170723014      -11.34       -5.66    1.0    170ms
 10   -2.772170723015      -12.09       -6.10    2.0    196ms
 11   -2.772170723015      -13.11       -6.54    1.0    191ms
 12   -2.772170723015      -15.35       -7.00    1.0    166ms
 13   -2.772170723015      -13.82       -7.48    2.0    205ms
 14   -2.772170723015   +  -14.18       -7.84    1.0    169ms
 15   -2.

1.77355804500281

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770441989222                   -0.53    8.0         
  2   -2.772046719761       -2.79       -1.29    1.0    203ms
  3   -2.772082163233       -4.45       -2.71    1.0    153ms
  4   -2.772083413432       -5.90       -3.83    2.0    227ms
  5   -2.772083417542       -8.39       -4.32    2.0    177ms
  6   -2.772083417804       -9.58       -5.69    1.0    181ms
  7   -2.772083417810      -11.18       -5.89    2.0    181ms
  8   -2.772083417811      -12.73       -6.69    1.0    175ms
  9   -2.772083417811   +  -14.27       -7.39    2.0    195ms
 10   -2.772083417811      -14.10       -7.79    1.0    184ms
 11   -2.772083417811      -14.88       -8.66    1.0    168ms

Polarizability via ForwardDiff:       1.7725349768872494
Polarizability via finite difference: 1.77355804500281
